In [4]:
import os, sys
import subprocess
from utilities_pipeline_status import *
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager_v2 import DataManager
from a_driver_utilities import *

from a_GUI_utilities_pipeline_status import *

print all_stacks

['MD6635TIFF', 'DK17CZI', 'DK23', 'DK19']


In [6]:
stack = 'DK23'

pipeline_status = get_pipeline_status( stack )
print(pipeline_status)

a_script_preprocess_3


In [85]:
def apply_transform( stack, T, input_fp, output_fp=None ):
    """
    Applies transform T onto the image at img_fp and return the result
    """
    
    #version = get_version(stack)
    
    #img_fp = DataManager.get_image_filepath_v2(stack=stack, section=metadata_cache['filenames_to_sections'][stack][fn], 
                            #prep_id='None', resol='thumbnail', version=version)
    img_fp = input_fp
    
    op_str = ''
    op_str += " +distort AffineProjection '%(sx)f,%(rx)f,%(ry)f,%(sy)f,%(tx)f,%(ty)f' " % {
     'sx':T[0,0],
     'sy':T[1,1],
     'rx':T[1,0],
     'ry':T[0,1],
     'tx':T[0,2],
     'ty':T[1,2],}
    
    x = 0
    y = 0
    w = 2000
    h = 1000
    op_str += ' -crop %(w)sx%(h)s%(x)s%(y)s\! ' % {
                     'x': '+' + str(x) if int(x) >= 0 else str(x),
                     'y': '+' + str(y) if int(y) >= 0 else str(y),
                     'w': str(w),
                     'h': str(h)}
    
    if output_fp==None:
        output_fp_root = os.path.join( os.environ['ROOT_DIR'], 'CSHL_data_processed', stack, 'tmp')
        if not os.path.exists( output_fp_root ):
            os.makedirs(output_fp_root)
        output_fp = os.path.join( output_fp_root, 'tmp_'+fn+'.tif' )
        delete_after = True
    else:
        if not os.path.exists( os.path.dirname(output_fp) ):
            os.makedirs( os.path.dirname(output_fp) )
        delete_after = False

    try:
        bg_color = 'black' #get_padding_color(stack)
        
        execute_command("convert \"%(input_fp)s\"  +repage -virtual-pixel background \
-background %(bg_color)s %(op_str)s -flatten -compress lzw \"%(output_fp)s\"" % \
                    {'op_str': op_str,
                     'input_fp': img_fp,
                     'output_fp': output_fp,
                     'bg_color': bg_color})
        
        img = cv2.imread( output_fp )
        if delete_after:
            os.remove( output_fp ) 
            pass
        return img
    except Exception as e:
        print e
        return None
    
def get_anchor_transform( stack, fn, downsampled=True ):
    if downsampled:
        T = DataManager.load_transforms(stack, downsample_factor=32, use_inverse=True)[fn]
    else:
        T = DataManager.load_transforms(stack, use_inverse=True)[fn]
    return T

In [16]:
for fn in metadata_cache['filenames_to_sections'][stack].keys():
    T = get_anchor_transform(stack, fn, downsampled=True)

In [67]:
5730./29

197.58620689655172

In [69]:
for fn in metadata_cache['filenames_to_sections'][stack].keys()[200:205]:
    fn_secondary_C = fn.replace('C0','C1').replace('CFP','TuYFP') + '_raw.tif'
    
    fp_secondary_C = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_C0', fn_secondary_C)
    fp_secondary_C_thumbnail = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_thumbnail_C0', fn_secondary_C)
    fp_secondary_C_output = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_prep1_C0_test', fn_secondary_C)
    
    # Load anchor transform
    T = get_anchor_transform(stack, fn, downsampled=True)
    
    apply_transform( stack, T, input_fp=fp_secondary_C_thumbnail, output_fp=fp_secondary_C_output )
    

convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide007_2019_09_10_S07_S6_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.995862,-0.090878,0.090878,0.995862,60.254510,35.940622'  -crop 2000x1000+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_C0_test/DK23_slide007_2019_09_10_S07_S6_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide025_2019_09_10_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999983,0.005889,-0.005889,0.999983,39.322505,7.197588'  -crop 2000x1000+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_C0_test/DK23_slide025_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_dat

In [107]:
fn_to_raw_T_dict = DataManager.load_transforms_v2(stack, in_image_resolution='raw', out_image_resolution='raw', 
                               use_inverse=False, anchor_fn=metadata_cache['anchor_fn'][stack])

fn_to_raw_T_dict = DataManager.load_transforms_v2(stack, in_image_resolution='thumbnail', out_image_resolution='thumbnail', 
                               use_inverse=True, anchor_fn=metadata_cache['anchor_fn'][stack])

for fn in fn_to_raw_T_dict.keys()[201:202]:
    fn_secondary_C = fn.replace('C0','C1').replace('CFP','TuYFP') + '_raw.tif'
    
    fp_secondary_C_raw = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_C0', fn_secondary_C)
    fp_secondary_C_thumbnail = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_thumbnail_C0', fn_secondary_C)
    fp_secondary_C_output = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_prep1_C0_test', fn_secondary_C)
    
    # Load anchor transform
    T = fn_to_raw_T_dict[fn]
#     T = np.array([[1,0,0],[0,1,0],[0,0,1]])
    
    apply_transform( stack, T, input_fp=fp_secondary_C_raw, output_fp=fp_secondary_C_output )
    

convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide042_2019_09_11_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999979,0.006413,-0.006413,0.999979,-136.270559,-8.321065'  -crop 2000x1000+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_C0_test/DK23_slide042_2019_09_11_S01_S0_C1_TuYFP_raw.tif"
return code: 0


In [111]:
img = cv2.imread( fp_secondary_C_raw )

In [112]:
np.shape(img)

(26007, 42574, 3)

In [113]:
img = img[:,:,0]

In [116]:
cv2.imwrite( '/data2/testImg.tif', img )

True

In [118]:
del img

In [117]:
T = np.array([[1,0,0],[0,1,0],[0,0,1]])
    
apply_transform( stack, T, input_fp='/data2/testImg.tif', output_fp=fp_secondary_C_output )

convert "/data2/testImg.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '1.000000,0.000000,0.000000,1.000000,0.000000,0.000000'  -crop 2000x1000+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_C0_test/DK23_slide042_2019_09_11_S01_S0_C1_TuYFP_raw.tif"
return code: 0


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [101]:
print get_anchor_transform(stack, fn, downsampled=32)

[[ 9.99979437e-01  6.41295604e-03  1.36321119e+02]
 [-6.41295604e-03  9.99979437e-01  7.44699663e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [90]:
DataManager.load_transforms_v2(stack, in_image_resolution='thumbnail', out_image_resolution='thumbnail', 
                               use_inverse=False, anchor_fn=metadata_cache['anchor_fn'][stack])[fn]

array([[ 9.99982660e-01, -5.88896596e-03,  2.45765654e+02],
       [ 5.88896596e-03,  9.99982660e-01,  4.49849244e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [94]:
DataManager.load_transforms_v2(stack, in_image_resolution='raw', out_image_resolution='raw', 
                               use_inverse=False, anchor_fn=metadata_cache['anchor_fn'][stack])[fn]

array([[ 9.99982660e-01, -5.88896596e-03,  7.86450093e+03],
       [ 5.88896596e-03,  9.99982660e-01,  1.43951758e+03],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [88]:
DataManager.load_transforms_v2(stack, in_image_resolution='thumbnail', out_image_resolution='thumbnail')[fn]

array([[ 9.99982660e-01,  5.88896596e-03, -2.46026307e+02],
       [-5.88896596e-03,  9.99982660e-01, -4.35368388e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [89]:
DataManager.load_transforms_v2(stack, in_image_resolution='thumbnail', out_image_resolution='thumbnail', anchor_fn=metadata_cache['anchor_fn'][stack])[fn]

array([[ 9.99982660e-01,  5.88896596e-03, -2.46026307e+02],
       [-5.88896596e-03,  9.99982660e-01, -4.35368388e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [91]:
229./5730

0.039965095986038396

In [93]:
1./32

0.03125

In [53]:
! python create_thumbnails_from_raw_images.py $stack

Setting environment for Precision WorkStation for Alex Newberry
No mcubes
[521 1057 128 465 None None]
[521 1057 128 465 None None]
[564 1088 102 736 None None]
[564 1088 102 736 None None]
[521 1057 128 465 None None]
[521 1057 128 465 None None]
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide039_2019_09_10_S15_S14_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide039_2019_09_10_S15_S14_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide083_2019_09_10_S15_S14_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide083_2019_09_10_S15_S14_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide073_2019_09_11_S15_S14_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide073_2019_09_11_S15_S14_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide063_2019_09_11_S15_S14_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide063_2019_09_11_S15_S14_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide117_2019_09_10_S12_S11_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide117_2019_09_10_S

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide093_2019_09_10_S01_S0_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide093_2019_09_10_S01_S0_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide023_2019_09_10_S19_S18_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide023_2019_09_10_S19_S18_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide044_2019_09_11_S08_S7_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide044_2019_09_11_S08_

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide029_2019_09_10_S15_S14_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide029_2019_09_10_S15_S14_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide054_2019_09_03_S08_S7_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide054_2019_09_03_S08_S7_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide106_2019_09_10_S19_S18_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide106_2019_09_10_S19

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide103_2019_09_10_S13_S12_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide103_2019_09_10_S13_S12_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide052_2019_09_11_S15_S14_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide052_2019_09_11_S15_S14_C0_CFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw/DK23_slide055_2019_09_11_S15_S14_C0_CFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail/DK23_slide055_2019_09_11_S

...

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide003_2019_09_10_S06_S5_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide003_2019_09_10_S06_S5_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide117_2019_09_10_S18_S17_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide117_2019_09_10_S18_S17_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide011_2019_09_10_S07_S6_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/D

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide023_2019_09_10_S01_S0_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide023_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide003_2019_09_10_S16_S15_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide003_2019_09_10_S16_S15_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide028_2019_09_10_S07_S6_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/D

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide003_2019_09_10_S11_S10_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide003_2019_09_10_S11_S10_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide030_2019_09_10_S08_S7_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide030_2019_09_10_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide081_2019_09_11_S08_S7_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/D

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide056_2019_09_11_S08_S7_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide056_2019_09_11_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide051_2019_09_11_S22_S21_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide051_2019_09_11_S22_S21_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide047_2019_09_11_S22_S21_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide084_2019_09_10_S15_S14_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide084_2019_09_10_S15_S14_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide105_2019_09_10_S01_S0_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide105_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide016_2019_09_10_S01_S0_C1_TuYFP_raw.tif" -resize 4% -auto-level -normalize                     -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/D

In [38]:
os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_thumbnail_C0', fn_secondary_C)

'/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide015_2019_09_10_S13_S12_C1_TuYFP_raw.tif'

In [37]:
! ls /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/

czi_raw_output
DK23_anchor.txt
DK23_custom_transforms
DK23_elastix_output
DK23_prep1_C0_test
DK23_prep1_thumbnail_NtbNormalized
DK23_raw
DK23_raw_C0
DK23_raw_Ntb
DK23_raw_thumbnail
DK23_raw_thumbnail_C0
DK23_sorted_filenames_post_slice_alignment.txt
DK23_sorted_filenames_till_slice_alignment.txt
DK23_sorted_filenames.txt
DK23_thumbnail_Ntb
DK23_thumbnail_NtbNormalized
DK23_transforms_to_anchor.csv
operation_configs
tmp


In [22]:
! mkdir /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_C0_test/

In [26]:
# This will hold the pointer to all 
valid_secondary_channel_fps = []
# Populate valid_secondary_channel_fps with valid fps
for i in range(5):
    secondary_channel_fp = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_C'+str(i))
    if os.path.exists(secondary_channel_fp):
        valid_secondary_channel_fps.append( secondary_channel_fp )

In [27]:
valid_secondary_channel_fps[0]

'/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0'

In [19]:
fp_to_check

'/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/raw_C4'

In [9]:
metadata_cache['filenames_to_sections'][stack]

{'DK23_slide001_2019_09_10_S05_S4_C0_CFP': 2,
 'DK23_slide001_2019_09_10_S09_S8_C0_CFP': 3,
 'DK23_slide002_2019_09_10_S01_S0_C0_CFP': 4,
 'DK23_slide002_2019_09_10_S06_S5_C0_CFP': 5,
 'DK23_slide002_2019_09_10_S11_S10_C0_CFP': 6,
 'DK23_slide002_2019_09_10_S16_S15_C0_CFP': 7,
 'DK23_slide003_2019_09_10_S01_S0_C0_CFP': 8,
 'DK23_slide003_2019_09_10_S06_S5_C0_CFP': 9,
 'DK23_slide003_2019_09_10_S11_S10_C0_CFP': 10,
 'DK23_slide004_2019_09_10_S01_S0_C0_CFP': 12,
 'DK23_slide004_2019_09_10_S06_S5_C0_CFP': 13,
 'DK23_slide004_2019_09_10_S11_S10_C0_CFP': 14,
 'DK23_slide004_2019_09_10_S16_S15_C0_CFP': 15,
 'DK23_slide005_2019_09_10_S01_S0_C0_CFP': 16,
 'DK23_slide005_2019_09_10_S06_S5_C0_CFP': 17,
 'DK23_slide005_2019_09_10_S11_S10_C0_CFP': 18,
 'DK23_slide005_2019_09_10_S17_S16_C0_CFP': 19,
 'DK23_slide006_2019_09_10_S01_S0_C0_CFP': 20,
 'DK23_slide006_2019_09_10_S06_S5_C0_CFP': 21,
 'DK23_slide006_2019_09_10_S12_S11_C0_CFP': 22,
 'DK23_slide006_2019_09_10_S18_S17_C0_CFP': 23,
 'DK23_slide

In [13]:
get_anchor_transform(stack, fn='DK23_slide060_2019_09_11_S08_S7_C0_CFP')

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
DataManager.